In [367]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [368]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%autoreload 2
%aimport AD_predictor_tools
%aimport AD_comparison_tools
%aimport PlottingTools

In [422]:
SFARI_tfs = pd.read_csv("../data/SFARI_TFs.csv")
SFARI_known_ADs = pd.read_csv("../output/SFARI_TF_known_ADs.csv", index_col = 0)

In [370]:
uniprotID_ENST_mapping_df = pd.read_csv("../data/SFARI_tf_ENST_codes.csv", index_col = 0)
uniprotID_ENST_mapping_df

,uniprotID,ENST
0,O00712,ENST00000380959.7
1,O14770,ENST00000561208.6
2,O15266,ENST00000381578.6
3,O15409,ENST00000350908.9
4,O43435,ENST00000329705.11
...,...,...
31,O95365,ENST00000322357.9
32,Q6ZMY9,ENST00000359971.4
33,Q8N859,ENST00000429591.4
34,Q6NX45,ENST00000354377.8


In [371]:
# Downloading the uniprot IDs to get the corresponding ENST codes
uniprotID_ENST_mapping_df[["ENST"]].to_csv("../data/SFARI_tf_ENST_codes.txt", 
                                                 header=None, index=None, sep=' ', mode='a')

Used ensembl's Biomart
1. Dataset: human genes GRCh38.p13
2. Filters: inputted the ENST codes from above
3. Attributes: under structures- chose all exon ones, also CDS length and gene name

In [402]:
# Multiple rows for each ENST id - corresponding to the exons
mart_output = pd.read_csv("../data/mart_export2.txt", sep = "\t")
mart_output

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Strand,Gene name,Chromosome/scaffold name,cDNA coding end,cDNA coding start,Exon region start (bp),Exon region end (bp),Constitutive exon,Exon rank in transcript,Start phase,End phase,Genomic coding start,Genomic coding end,Exon stable ID,CDS start,CDS end
0,ENSG00000010818,ENSG00000010818.11,ENST00000012134,ENST00000012134.7,-1,HIVEP2,6,NaN,NaN,142944801,142945031,0,1,-1,-1,NaN,NaN,ENSE00003804252,NaN,NaN
1,ENSG00000010818,ENSG00000010818.11,ENST00000012134,ENST00000012134.7,-1,HIVEP2,6,NaN,NaN,142836935,142837047,0,2,-1,-1,NaN,NaN,ENSE00001445148,NaN,NaN
2,ENSG00000010818,ENSG00000010818.11,ENST00000012134,ENST00000012134.7,-1,HIVEP2,6,NaN,NaN,142776147,142776191,0,3,-1,-1,NaN,NaN,ENSE00001445146,NaN,NaN
3,ENSG00000010818,ENSG00000010818.11,ENST00000012134,ENST00000012134.7,-1,HIVEP2,6,5963.0,777.0,142769552,142775125,0,4,-1,0,142769552.0,142774738.0,ENSE00001445111,1.0,5187.0
4,ENSG00000010818,ENSG00000010818.11,ENST00000012134,ENST00000012134.7,-1,HIVEP2,6,6118.0,5964.0,142768382,142768536,0,5,0,2,142768382.0,142768536.0,ENSE00000764841,5188.0,5342.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,ENSG00000181827,ENSG00000181827.16,ENST00000673997,ENST00000673997.1,-1,RFX7,15,1510.0,1215.0,56098081,56098376,0,8,1,0,56098081.0,56098376.0,ENSE00001386445,521.0,816.0
1400,ENSG00000181827,ENSG00000181827.16,ENST00000673997,ENST00000673997.1,-1,RFX7,15,NaN,NaN,56243445,56243879,0,1,-1,-1,NaN,NaN,ENSE00003897160,NaN,NaN
1401,ENSG00000181827,ENSG00000181827.16,ENST00000673997,ENST00000673997.1,-1,RFX7,15,4786.0,1511.0,56087280,56096620,0,9,0,-1,56093345.0,56096620.0,ENSE00002549465,817.0,4092.0
1402,ENSG00000181827,ENSG00000181827.16,ENST00000673997,ENST00000673997.1,-1,RFX7,15,NaN,NaN,56243125,56243287,0,2,-1,-1,NaN,NaN,ENSE00003896653,NaN,NaN


In [403]:
mart_output.columns

Index(['Gene stable ID', 'Gene stable ID version', 'Transcript stable ID',
       'Transcript stable ID version', 'Strand', 'Gene name',
       'Chromosome/scaffold name', 'cDNA coding end', 'cDNA coding start',
       'Exon region start (bp)', 'Exon region end (bp)', 'Constitutive exon',
       'Exon rank in transcript', 'Start phase', 'End phase',
       'Genomic coding start', 'Genomic coding end', 'Exon stable ID',
       'CDS start', 'CDS end'],
      dtype='object')

In [404]:
len(set(mart_output["Transcript stable ID version"]))

122

In [405]:
# Starting with AR to verify on one
AR_mart_output = mart_output[mart_output["Gene name"] == "AR"]
AR_mart_output

# Seems like genomic coding start and genomic coding end are the columns we're interested in

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Strand,Gene name,Chromosome/scaffold name,cDNA coding end,cDNA coding start,Exon region start (bp),Exon region end (bp),Constitutive exon,Exon rank in transcript,Start phase,End phase,Genomic coding start,Genomic coding end,Exon stable ID,CDS start,CDS end
832,ENSG00000169083,ENSG00000169083.18,ENST00000374690,ENST00000374690.9,1,AR,X,2742.0,1127.0,67544021,67546762,0,1,-1,2,67545147.0,67546762.0,ENSE00001326500,1.0,1616.0
833,ENSG00000169083,ENSG00000169083.18,ENST00000374690,ENST00000374690.9,1,AR,X,2894.0,2743.0,67643256,67643407,0,2,2,1,67643256.0,67643407.0,ENSE00003606052,1617.0,1768.0
834,ENSG00000169083,ENSG00000169083.18,ENST00000374690,ENST00000374690.9,1,AR,X,3011.0,2895.0,67686010,67686126,0,3,1,1,67686010.0,67686126.0,ENSE00003625235,1769.0,1885.0
835,ENSG00000169083,ENSG00000169083.18,ENST00000374690,ENST00000374690.9,1,AR,X,3299.0,3012.0,67711402,67711689,0,4,1,1,67711402.0,67711689.0,ENSE00001165476,1886.0,2173.0
836,ENSG00000169083,ENSG00000169083.18,ENST00000374690,ENST00000374690.9,1,AR,X,3444.0,3300.0,67717478,67717622,0,5,1,2,67717478.0,67717622.0,ENSE00001282597,2174.0,2318.0
837,ENSG00000169083,ENSG00000169083.18,ENST00000374690,ENST00000374690.9,1,AR,X,3575.0,3445.0,67721833,67721963,0,6,2,1,67721833.0,67721963.0,ENSE00001165458,2319.0,2449.0
838,ENSG00000169083,ENSG00000169083.18,ENST00000374690,ENST00000374690.9,1,AR,X,3733.0,3576.0,67722827,67722984,0,7,1,0,67722827.0,67722984.0,ENSE00001316881,2450.0,2607.0
839,ENSG00000169083,ENSG00000169083.18,ENST00000374690,ENST00000374690.9,1,AR,X,3889.0,3734.0,67723686,67730619,0,8,0,-1,67723686.0,67723841.0,ENSE00001930911,2608.0,2763.0


In [407]:
mart_output[mart_output["Gene stable ID"] == "ENSG00000118263"]

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Strand,Gene name,Chromosome/scaffold name,cDNA coding end,cDNA coding start,Exon region start (bp),Exon region end (bp),Constitutive exon,Exon rank in transcript,Start phase,End phase,Genomic coding start,Genomic coding end,Exon stable ID,CDS start,CDS end
322,ENSG00000118263,ENSG00000118263.16,ENST00000309446,ENST00000309446.11,-1,KLF7,2,482.0,381.0,207165467,207165948,0,1,-1,0,207165467.0,207165568.0,ENSE00001401308,1.0,102.0
323,ENSG00000118263,ENSG00000118263.16,ENST00000309446,ENST00000309446.11,-1,KLF7,2,1113.0,483.0,207123774,207124404,0,2,0,1,207123774.0,207124404.0,ENSE00000934675,103.0,733.0
324,ENSG00000118263,ENSG00000118263.16,ENST00000309446,ENST00000309446.11,-1,KLF7,2,1237.0,1114.0,207088458,207088581,0,3,1,2,207088458.0,207088581.0,ENSE00003473020,734.0,857.0
325,ENSG00000118263,ENSG00000118263.16,ENST00000309446,ENST00000309446.11,-1,KLF7,2,1289.0,1238.0,207074137,207081264,0,4,2,-1,207081213.0,207081264.0,ENSE00001854866,858.0,909.0


In [408]:
from Bio.Seq import Seq


In [414]:
seq = Seq("TCGGGCCCAAAAAAA")


In [415]:
print(seq.translate())

SGPKK


In [417]:
print(seq.reverse_complement().translate())

FFWAR


In [420]:
str(seq.translate())

'SGPKK'

In [944]:
relevant_AR_mart_output = AR_mart_output[["Chromosome/scaffold name", \
                                          "Genomic coding start", "Genomic coding end", \
                                         "CDS start", "CDS end", "Exon rank in transcript"]]
relevant_AR_mart_output["Genomic coding start"] = relevant_AR_mart_output["Genomic coding start"].astype(int)
relevant_AR_mart_output["Genomic coding end"] = relevant_AR_mart_output["Genomic coding end"].astype(int)
relevant_AR_mart_output = relevant_AR_mart_output.sort_values(by = "Exon rank in transcript")
relevant_AR_mart_output

,Chromosome/scaffold name,Genomic coding start,Genomic coding end,CDS start,CDS end,Exon rank in transcript
832,X,67545147,67546762,1.0,1616.0,1
833,X,67643256,67643407,1617.0,1768.0,2
834,X,67686010,67686126,1769.0,1885.0,3
835,X,67711402,67711689,1886.0,2173.0,4
836,X,67717478,67717622,2174.0,2318.0,5
837,X,67721833,67721963,2319.0,2449.0,6
838,X,67722827,67722984,2450.0,2607.0,7
839,X,67723686,67723841,2608.0,2763.0,8


In [945]:
AR_AD_rows = SFARI_known_ADs[SFARI_known_ADs["Gene"] == "AR"]
AR_AD_rows

,Gene,Start,End,uniprotID,Reference,ProteinSeq
7,AR,652,910,P10275,"PMID: 9033395, Soto",PTEETTQKLTVSHIEGYECQPIFLNVLEAIEPGVVCAGHDNNQPDS...
8,AR,1,494,P10275,"PMID: 9033395, 7706276, Soto",MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...


In [946]:
AA_start = 652
AA_end = 910

In [947]:
bp_start = (AA_start - 1) * 3
bp_end = AA_end * 3

In [948]:
bp_start

1953

In [949]:
bp_end

2730

In [950]:
filtered_AR_mart_output = relevant_AR_mart_output[(relevant_AR_mart_output["CDS end"] > bp_start) \
                       & (relevant_AR_mart_output["CDS start"] < bp_end)]
filtered_AR_mart_output = filtered_AR_mart_output.reset_index(drop = True)
filtered_AR_mart_output

,Chromosome/scaffold name,Genomic coding start,Genomic coding end,CDS start,CDS end,Exon rank in transcript
0,X,67711402,67711689,1886.0,2173.0,4
1,X,67717478,67717622,2174.0,2318.0,5
2,X,67721833,67721963,2319.0,2449.0,6
3,X,67722827,67722984,2450.0,2607.0,7
4,X,67723686,67723841,2608.0,2763.0,8


In [951]:
# Adjust first row
add = (bp_start - filtered_AR_mart_output["CDS start"].iloc[0] + 1)
add

68.0

In [952]:
filtered_AR_mart_output.at[0, "Genomic coding start"] += add
filtered_AR_mart_output.at[0, "CDS start"] += add

In [953]:
filtered_AR_mart_output

,Chromosome/scaffold name,Genomic coding start,Genomic coding end,CDS start,CDS end,Exon rank in transcript
0,X,67711470,67711689,1954.0,2173.0,4
1,X,67717478,67717622,2174.0,2318.0,5
2,X,67721833,67721963,2319.0,2449.0,6
3,X,67722827,67722984,2450.0,2607.0,7
4,X,67723686,67723841,2608.0,2763.0,8


In [954]:
# Adjust last row
subtract = filtered_AR_mart_output["CDS end"].iloc[-1] - bp_end
subtract

33.0

In [955]:
last_index = filtered_AR_mart_output.index[-1]
filtered_AR_mart_output.at[last_index, "Genomic coding end"] -= subtract
filtered_AR_mart_output.at[last_index, "CDS end"] -= subtract
filtered_AR_mart_output

,Chromosome/scaffold name,Genomic coding start,Genomic coding end,CDS start,CDS end,Exon rank in transcript
0,X,67711470,67711689,1954.0,2173.0,4
1,X,67717478,67717622,2174.0,2318.0,5
2,X,67721833,67721963,2319.0,2449.0,6
3,X,67722827,67722984,2450.0,2607.0,7
4,X,67723686,67723808,2608.0,2730.0,8


---
Everything below is just for validation

In [956]:
# Source for following lines: https://stackoverflow.com/questions/58010068/how-to-explode-range-from-two-column-to-rows
zipped = zip(filtered_AR_mart_output['Chromosome/scaffold name'],
             filtered_AR_mart_output['Genomic coding start'], 
            filtered_AR_mart_output['Genomic coding end'])
genomic_coord_df = pd.DataFrame([(i, y) for i,j,k in zipped for y in range(j, k+1)], columns=['chr', 'position'])
# genomic_coord_df = genomic_coord_df.iloc[:-1, :]
genomic_coord_df

,chr,position
0,X,67711470
1,X,67711471
2,X,67711472
3,X,67711473
4,X,67711474
...,...,...
772,X,67723804
773,X,67723805
774,X,67723806
775,X,67723807


In [957]:
# indices = genomic_coord_df.loc[range(bp_start, bp_end)]
# indices = list(indices["position"] - 1)
indices = list(genomic_coord_df["position"] - 1)
indices

[67711469,
 67711470,
 67711471,
 67711472,
 67711473,
 67711474,
 67711475,
 67711476,
 67711477,
 67711478,
 67711479,
 67711480,
 67711481,
 67711482,
 67711483,
 67711484,
 67711485,
 67711486,
 67711487,
 67711488,
 67711489,
 67711490,
 67711491,
 67711492,
 67711493,
 67711494,
 67711495,
 67711496,
 67711497,
 67711498,
 67711499,
 67711500,
 67711501,
 67711502,
 67711503,
 67711504,
 67711505,
 67711506,
 67711507,
 67711508,
 67711509,
 67711510,
 67711511,
 67711512,
 67711513,
 67711514,
 67711515,
 67711516,
 67711517,
 67711518,
 67711519,
 67711520,
 67711521,
 67711522,
 67711523,
 67711524,
 67711525,
 67711526,
 67711527,
 67711528,
 67711529,
 67711530,
 67711531,
 67711532,
 67711533,
 67711534,
 67711535,
 67711536,
 67711537,
 67711538,
 67711539,
 67711540,
 67711541,
 67711542,
 67711543,
 67711544,
 67711545,
 67711546,
 67711547,
 67711548,
 67711549,
 67711550,
 67711551,
 67711552,
 67711553,
 67711554,
 67711555,
 67711556,
 67711557,
 67711558,
 67711559,

In [958]:
# Downloading hg38 data from UCSC genome browser
# ! rsync -avzP rsync://hgdownload.cse.ucsc.edu/goldenPath/hg38/chromosomes/ ../data/

In [959]:
# ! gunzip ../data/hg38/*

In [960]:
chrX = AD_predictor_tools.makeFullLengthProteinDF("../data/hg38/chrX.fa")
chrX

There are 1 proteins


,GeneName,AAseq
0,chrX,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...


In [961]:
AR_DNA_seq = ""
for index in indices:
    AR_DNA_seq += chrX["AAseq"].iloc[0][index]

In [962]:
AR_DNA_seq

'CCCACTGAGGAGACAACCCAGAAGCTGACAGTGTCACACATTGAAGGCTATGAATGTCAGCCCATCTTTCTGAATGTCCTGGAAGCCATTGAGCCAGGTGTAGTGTGTGCTGGACACGACAACAACCAGCCCGACTCCTTTGCAGCCTTGCTCTCTAGCCTCAATGAACTGGGAGAGAGACAGCTTGTACACGTGGTCAAGTGGGCCAAGGCCTTGCCTGGCTTCCGCAACTTACACGTGGACGACCAGATGGCTGTCATTCAGTACTCCTGGATGGGGCTCATGGTGTTTGCCATGGGCTGGCGATCCTTCACCAATGTCAACTCCAGGATGCTCTACTTCGCCCCTGATCTGGTTTTCAATGAGTACCGCATGCACAAGTCCCGGATGTACAGCCAGTGTGTCCGAATGAGGCACCTCTCTCAAGAGTTTGGATGGCTCCAAATCACCCCCCAGGAATTCCTGTGCATGAAAGCACTGCTACTCTTCAGCATTATTCCAGTGGATGGGCTGAAAAATCAAAAATTCTTTGATGAACTTCGAATGAACTACATCAAGGAACTCGATCGTATCATTGCATGCAAAAGAAAAAATCCCACATCCTGCTCAAGACGCTTCTACCAGCTCACCAAGCTCCTGGACTCCGTGCAGCCTATTGCGAGAGAGCTGCATCAGTTCACTTTTGACCTGCTAATCAAGTCACACATGGTGAGCGTGGACTTTCCGGAAATGATGGCAGAGATCATCTCTGTGCAAGTGCCCAAGATCCTTTCTGGG'

In [963]:
# From https://www.geeksforgeeks.org/dna-protein-python-3/

def translate(seq):
      
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    protein = ""
    if len(seq)%3 == 0:
        for i in range(0, len(seq), 3):
            codon = seq[i:i + 3]
            protein+= table[codon]
    else:
        print(len(seq))
    return protein

In [964]:
AR_DNA_seq

'CCCACTGAGGAGACAACCCAGAAGCTGACAGTGTCACACATTGAAGGCTATGAATGTCAGCCCATCTTTCTGAATGTCCTGGAAGCCATTGAGCCAGGTGTAGTGTGTGCTGGACACGACAACAACCAGCCCGACTCCTTTGCAGCCTTGCTCTCTAGCCTCAATGAACTGGGAGAGAGACAGCTTGTACACGTGGTCAAGTGGGCCAAGGCCTTGCCTGGCTTCCGCAACTTACACGTGGACGACCAGATGGCTGTCATTCAGTACTCCTGGATGGGGCTCATGGTGTTTGCCATGGGCTGGCGATCCTTCACCAATGTCAACTCCAGGATGCTCTACTTCGCCCCTGATCTGGTTTTCAATGAGTACCGCATGCACAAGTCCCGGATGTACAGCCAGTGTGTCCGAATGAGGCACCTCTCTCAAGAGTTTGGATGGCTCCAAATCACCCCCCAGGAATTCCTGTGCATGAAAGCACTGCTACTCTTCAGCATTATTCCAGTGGATGGGCTGAAAAATCAAAAATTCTTTGATGAACTTCGAATGAACTACATCAAGGAACTCGATCGTATCATTGCATGCAAAAGAAAAAATCCCACATCCTGCTCAAGACGCTTCTACCAGCTCACCAAGCTCCTGGACTCCGTGCAGCCTATTGCGAGAGAGCTGCATCAGTTCACTTTTGACCTGCTAATCAAGTCACACATGGTGAGCGTGGACTTTCCGGAAATGATGGCAGAGATCATCTCTGTGCAAGTGCCCAAGATCCTTTCTGGG'

In [965]:
translate(AR_DNA_seq)

'PTEETTQKLTVSHIEGYECQPIFLNVLEAIEPGVVCAGHDNNQPDSFAALLSSLNELGERQLVHVVKWAKALPGFRNLHVDDQMAVIQYSWMGLMVFAMGWRSFTNVNSRMLYFAPDLVFNEYRMHKSRMYSQCVRMRHLSQEFGWLQITPQEFLCMKALLLFSIIPVDGLKNQKFFDELRMNYIKELDRIIACKRKNPTSCSRRFYQLTKLLDSVQPIARELHQFTFDLLIKSHMVSVDFPEMMAEIISVQVPKILSG'

In [966]:
SFARI_known_ADs[SFARI_known_ADs["Gene"] == "AR"]

,Gene,Start,End,uniprotID,Reference,ProteinSeq
7,AR,652,910,P10275,"PMID: 9033395, Soto",PTEETTQKLTVSHIEGYECQPIFLNVLEAIEPGVVCAGHDNNQPDS...
8,AR,1,494,P10275,"PMID: 9033395, 7706276, Soto",MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...


In [968]:
SFARI_known_ADs[SFARI_known_ADs["Gene"] == "AR"]["ProteinSeq"].iloc[0]

'PTEETTQKLTVSHIEGYECQPIFLNVLEAIEPGVVCAGHDNNQPDSFAALLSSLNELGERQLVHVVKWAKALPGFRNLHVDDQMAVIQYSWMGLMVFAMGWRSFTNVNSRMLYFAPDLVFNEYRMHKSRMYSQCVRMRHLSQEFGWLQITPQEFLCMKALLLFSIIPVDGLKNQKFFDELRMNYIKELDRIIACKRKNPTSCSRRFYQLTKLLDSVQPIARELHQFTFDLLIKSHMVSVDFPEMMAEIISVQVPKILSG'

In [ ]:
# The translated sequence matches!!

---
Now, trying with a negative strand sequence

In [851]:
SFARI_known_ADs[SFARI_known_ADs["Gene"] == "MEIS2"]

,Gene,Start,End,uniprotID,Reference,ProteinSeq
0,MEIS2,340,477,O14770,"activation_regions.txt, GSL",DQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQ...


In [852]:
# Starting with AR to verify on one
MEIS2_mart_output = mart_output[mart_output["Gene name"] == "MEIS2"]
MEIS2_mart_output

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Strand,Gene name,Chromosome/scaffold name,cDNA coding end,cDNA coding start,Exon region start (bp),Exon region end (bp),Constitutive exon,Exon rank in transcript,Start phase,End phase,Genomic coding start,Genomic coding end,Exon stable ID,CDS start,CDS end
1295,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,1095.0,1084.0,37099455,37100549,0,1,-1,0,37099455.0,37099466.0,ENSE00001929803,1.0,12.0
1296,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,1328.0,1096.0,37097967,37098199,0,2,0,2,37097967.0,37098199.0,ENSE00003684036,13.0,245.0
1297,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,1470.0,1329.0,37096289,37096430,0,3,2,0,37096289.0,37096430.0,ENSE00003594817,246.0,387.0
1298,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,1521.0,1471.0,37095564,37095614,0,4,0,0,37095564.0,37095614.0,ENSE00003642793,388.0,438.0
1299,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,1572.0,1522.0,37094527,37094577,0,5,0,0,37094527.0,37094577.0,ENSE00003759114,439.0,489.0
1300,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,1722.0,1573.0,37093581,37093730,0,6,0,0,37093581.0,37093730.0,ENSE00003565296,490.0,639.0
1301,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,1837.0,1723.0,37083771,37083885,0,7,0,1,37083771.0,37083885.0,ENSE00003699736,640.0,754.0
1302,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,1983.0,1838.0,37036814,37036959,0,8,1,0,37036814.0,37036959.0,ENSE00003544519,755.0,900.0
1303,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2060.0,1984.0,36950324,36950400,0,9,0,2,36950324.0,36950400.0,ENSE00003601407,901.0,977.0
1304,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2119.0,2061.0,36896628,36896686,0,10,2,1,36896628.0,36896686.0,ENSE00003626491,978.0,1036.0


In [870]:
AA_start = 340
AA_end = 477

bp_start = (AA_start - 1) * 3
bp_end = AA_end * 3

In [871]:
bp_start

1017

In [872]:
bp_end

1431

In [873]:
filtered_MEIS2_mart_output = MEIS2_mart_output[(MEIS2_mart_output["CDS end"] > bp_start) \
                       & (MEIS2_mart_output["CDS start"] < bp_end)]
filtered_MEIS2_mart_output = filtered_MEIS2_mart_output.reset_index(drop = True)
filtered_MEIS2_mart_output

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Strand,Gene name,Chromosome/scaffold name,cDNA coding end,cDNA coding start,Exon region start (bp),Exon region end (bp),Constitutive exon,Exon rank in transcript,Start phase,End phase,Genomic coding start,Genomic coding end,Exon stable ID,CDS start,CDS end
0,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2119.0,2061.0,36896628,36896686,0,10,2,1,36896628.0,36896686.0,ENSE00003626491,978.0,1036.0
1,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2230.0,2120.0,36895151,36895261,0,11,1,1,36895151.0,36895261.0,ENSE00003487762,1037.0,1147.0
2,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2517.0,2231.0,36889204,36892459,0,12,1,-1,36892173.0,36892459.0,ENSE00003728167,1148.0,1434.0


In [874]:
# Adjust the first row
subtract = bp_start - filtered_MEIS2_mart_output["CDS start"].iloc[0] + 1
filtered_MEIS2_mart_output.at[0, "Genomic coding end"] -= subtract

# Adjust the last row
add = filtered_MEIS2_mart_output["CDS end"].iloc[-1] - bp_end
last_index = filtered_MEIS2_mart_output.index[-1]
filtered_MEIS2_mart_output.at[last_index, "Genomic coding start"] += add

In [875]:
filtered_MEIS2_mart_output = filtered_MEIS2_mart_output.sort_values(by = "Genomic coding start")
filtered_MEIS2_mart_output

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Strand,Gene name,Chromosome/scaffold name,cDNA coding end,cDNA coding start,Exon region start (bp),Exon region end (bp),Constitutive exon,Exon rank in transcript,Start phase,End phase,Genomic coding start,Genomic coding end,Exon stable ID,CDS start,CDS end
2,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2517.0,2231.0,36889204,36892459,0,12,1,-1,36892176.0,36892459.0,ENSE00003728167,1148.0,1434.0
1,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2230.0,2120.0,36895151,36895261,0,11,1,1,36895151.0,36895261.0,ENSE00003487762,1037.0,1147.0
0,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2119.0,2061.0,36896628,36896686,0,10,2,1,36896628.0,36896646.0,ENSE00003626491,978.0,1036.0


In [876]:
filtered_MEIS2_mart_output["Genomic coding start"] = filtered_MEIS2_mart_output["Genomic coding start"].astype(int)
filtered_MEIS2_mart_output["Genomic coding end"] = filtered_MEIS2_mart_output["Genomic coding end"].astype(int)

In [877]:
filtered_MEIS2_mart_output

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Strand,Gene name,Chromosome/scaffold name,cDNA coding end,cDNA coding start,Exon region start (bp),Exon region end (bp),Constitutive exon,Exon rank in transcript,Start phase,End phase,Genomic coding start,Genomic coding end,Exon stable ID,CDS start,CDS end
2,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2517.0,2231.0,36889204,36892459,0,12,1,-1,36892176,36892459,ENSE00003728167,1148.0,1434.0
1,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2230.0,2120.0,36895151,36895261,0,11,1,1,36895151,36895261,ENSE00003487762,1037.0,1147.0
0,ENSG00000134138,ENSG00000134138.22,ENST00000561208,ENST00000561208.6,-1,MEIS2,15,2119.0,2061.0,36896628,36896686,0,10,2,1,36896628,36896646,ENSE00003626491,978.0,1036.0


In [878]:
# Source for following lines: https://stackoverflow.com/questions/58010068/how-to-explode-range-from-two-column-to-rows
zipped = zip(filtered_MEIS2_mart_output['Chromosome/scaffold name'],
             filtered_MEIS2_mart_output['Genomic coding start'], 
            filtered_MEIS2_mart_output['Genomic coding end'])
genomic_coord_df = pd.DataFrame([(i, y) for i,j,k in zipped for y in range(j, k+1)], columns=['chr', 'position'])
genomic_coord_df

,chr,position
0,15,36892176
1,15,36892177
2,15,36892178
3,15,36892179
4,15,36892180
...,...,...
409,15,36896642
410,15,36896643
411,15,36896644
412,15,36896645


In [879]:
# indices = genomic_coord_df.loc[range(bp_start, bp_end)]
# indices = list(indices["position"] - 1)
indices = list(genomic_coord_df["position"] - 1)

In [880]:
chr15 = AD_predictor_tools.makeFullLengthProteinDF("../data/hg38/chr15.fa")
chr15

There are 1 proteins


,GeneName,AAseq
0,chr15,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...


In [881]:
MEIS2_DNA_seq = ""
for index in indices:
    MEIS2_DNA_seq += chr15["AAseq"].iloc[0][index]

In [882]:
MEIS2_DNA_seq

'TTGGGCATGAATGTCCATAACCTGTCCGCCAACATTGGGATCTACAGAATTTAACATTGTGGGGCTCTGTGCTGACATAGTCATTCCAGGGTGGGTAGGGGGTCCTCCGTGCATCATCATGGCTGGGTGGTGGGGATGGCTTGGCAAATATGAATGCATTGGGGGTCCATGTCTTAATTGAGTAGGGTGTGGGGTCATCTGGGGAGGAGTGTAACTTGGCTGTGCCATACTCATTCCCATAGGACCACCCTGAGAAACGTAGTCCCCTGGCATGCTCTGCAAACCTGCAGGCCGGATCCCCATGTGTTGCTGACCATCCAACACAAAGCTCCCCATGGGCTGACCCTCTGGACTATATGCTGCTCCTTGGCTCACTGAAGGATCAAGAAGAAAACCTGCTCGATTTGACTGGTC'

In [883]:
seq = Seq(MEIS2_DNA_seq)

In [884]:
str(seq.reverse_complement())

'GACCAGTCAAATCGAGCAGGTTTTCTTCTTGATCCTTCAGTGAGCCAAGGAGCAGCATATAGTCCAGAGGGTCAGCCCATGGGGAGCTTTGTGTTGGATGGTCAGCAACACATGGGGATCCGGCCTGCAGGTTTGCAGAGCATGCCAGGGGACTACGTTTCTCAGGGTGGTCCTATGGGAATGAGTATGGCACAGCCAAGTTACACTCCTCCCCAGATGACCCCACACCCTACTCAATTAAGACATGGACCCCCAATGCATTCATATTTGCCAAGCCATCCCCACCACCCAGCCATGATGATGCACGGAGGACCCCCTACCCACCCTGGAATGACTATGTCAGCACAGAGCCCCACAATGTTAAATTCTGTAGATCCCAATGTTGGCGGACAGGTTATGGACATTCATGCCCAA'

In [885]:
str(seq.reverse_complement().translate())

'DQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQSMPGDYVSQGGPMGMSMAQPSYTPPQMTPHPTQLRHGPPMHSYLPSHPHHPAMMMHGGPPTHPGMTMSAQSPTMLNSVDPNVGGQVMDIHAQ'

In [706]:
SFARI_known_ADs[SFARI_known_ADs["Gene"] == "MEIS2"]["ProteinSeq"].iloc[0]

'DQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQSMPGDYVSQGGPMGMSMAQPSYTPPQMTPHPTQLRHGPPMHSYLPSHPHHPAMMMHGGPPTHPGMTMSAQSPTMLNSVDPNVGGQVMDIHAQ'

In [ ]:
# Matches!!